# import packages & library

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import time
from tqdm import tqdm

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}
TIMEOUT = 30

def get_book_metadata(book_id):
    try:
        base_url = f"https://www.gutenberg.org/ebooks/{book_id}"
        response = requests.get(base_url, headers=HEADERS, timeout=TIMEOUT)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        title = soup.find('h1', {'itemprop': 'name'}).get_text(strip=True) if soup.find('h1', {'itemprop': 'name'}) else f"Untitled {book_id}"
        author = soup.find('a', {'itemprop': 'creator'}).get_text(strip=True) if soup.find('a', {'itemprop': 'creator'}) else "Unknown"
        
        cover_img = soup.find('img', {'itemprop': 'image'})
        if cover_img:
            cover_url = cover_img['src']
            if not cover_url.startswith('http'):
                cover_url = f"https://www.gutenberg.org{cover_url}"
        else:
            cover_url = f"https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.cover.medium.jpg"

        return title, author, cover_url
    except Exception as e:
        print(f"❌ Error fetching metadata for {book_id}: {str(e)}")
        return None, None, None

def get_book_text(book_id):
    text_urls = [
        f"https://www.gutenberg.org/files/{book_id}/{book_id}-0.txt",
        f"https://www.gutenberg.org/files/{book_id}/{book_id}.txt",
        f"https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.txt"
    ]
    
    for url in text_urls:
        try:
            response = requests.get(url, headers=HEADERS, timeout=TIMEOUT)
            if response.status_code == 200:
                try:
                    return response.content.decode('utf-8')
                except UnicodeDecodeError:
                    try:
                        return response.content.decode('latin-1')
                    except UnicodeDecodeError:
                        return None
        except:
            continue
    return None

def extract_main_text(raw_text):
    if not raw_text:
        return None
    start_marker = '*** START OF THIS PROJECT GUTENBERG EBOOK'
    end_marker = '*** END OF THIS PROJECT GUTENBERG EBOOK'
    start = raw_text.upper().find(start_marker)
    end = raw_text.upper().find(end_marker)
    if start != -1:
        raw_text = raw_text[start + len(start_marker):]
    if end != -1:
        raw_text = raw_text[:end]
    return raw_text.strip()

def clean_text(text):
    if not text:
        return ""
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\[\d+\]', '', text)
    return text.strip()[:500000]

def save_to_csv(books, filename="books_data.csv"):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ID', 'Title', 'Author', 'Cover', 'Text'])
        for book in books:
            writer.writerow(book)
    print(f"\n📚 تم حفظ {len(books)} كتاب في الملف: {filename}")

# تحميل 10 كتب
books = []

for book_id in tqdm(range(1, 11), desc="📖 تحميل الكتب"):
    title, author, cover_url = get_book_metadata(book_id)
    if not title:
        continue

    raw_text = get_book_text(book_id)
    if not raw_text:
        continue

    main_text = extract_main_text(raw_text)
    cleaned = clean_text(main_text)

    if not cleaned:
        continue

    books.append([book_id, title, author, cover_url, cleaned])
    time.sleep(2)

# حفظهم في CSV
save_to_csv(books)


📖 تحميل الكتب:  60%|██████    | 6/10 [02:14<01:29, 22.35s/it]

❌ Error fetching metadata for 6: HTTPSConnectionPool(host='www.gutenberg.org', port=443): Max retries exceeded with url: /ebooks/6 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001A529EE7610>, 'Connection to www.gutenberg.org timed out. (connect timeout=30)'))


📖 تحميل الكتب:  70%|███████   | 7/10 [02:35<01:05, 21.92s/it]

❌ Error fetching metadata for 7: HTTPSConnectionPool(host='www.gutenberg.org', port=443): Max retries exceeded with url: /ebooks/7 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001A529EE7ED0>, 'Connection to www.gutenberg.org timed out. (connect timeout=30)'))


📖 تحميل الكتب: 100%|██████████| 10/10 [02:48<00:00, 16.82s/it]


📚 تم حفظ 8 كتاب في الملف: books_data.csv


In [5]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import time
from tqdm import tqdm

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}
TIMEOUT = 30

def get_book_metadata(book_id):
    try:
        base_url = f"https://www.gutenberg.org/ebooks/{book_id}"
        response = requests.get(base_url, headers=HEADERS, timeout=TIMEOUT)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        title = soup.find('h1', {'itemprop': 'name'}).get_text(strip=True) if soup.find('h1', {'itemprop': 'name'}) else f"Untitled {book_id}"
        author = soup.find('a', {'itemprop': 'creator'}).get_text(strip=True) if soup.find('a', {'itemprop': 'creator'}) else "Unknown"
        
        # البحث عن الملخص
        summary = ""
        for heading in soup.find_all(['h2', 'h3']):
            if 'summary' in heading.get_text().lower() or 'about' in heading.get_text().lower():
                next_node = heading.find_next_sibling()
                while next_node and next_node.name not in ['h2', 'h3']:
                    if next_node.name == 'p':
                        summary += next_node.get_text() + " "
                    next_node = next_node.find_next_sibling()
                break
        summary = summary.strip() or "No summary available"
        
        cover_img = soup.find('img', {'itemprop': 'image'})
        if cover_img:
            cover_url = cover_img['src']
            if not cover_url.startswith('http'):
                cover_url = f"https://www.gutenberg.org{cover_url}"
        else:
            cover_url = f"https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.cover.medium.jpg"

        return title, author, summary, cover_url
    except Exception as e:
        print(f"❌ Error fetching metadata for {book_id}: {str(e)}")
        return None, None, None, None

def get_book_text(book_id):
    text_urls = [
        f"https://www.gutenberg.org/files/{book_id}/{book_id}-0.txt",
        f"https://www.gutenberg.org/files/{book_id}/{book_id}.txt",
        f"https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.txt"
    ]
    
    for url in text_urls:
        try:
            response = requests.get(url, headers=HEADERS, timeout=TIMEOUT)
            if response.status_code == 200:
                try:
                    return response.content.decode('utf-8')
                except UnicodeDecodeError:
                    try:
                        return response.content.decode('latin-1')
                    except UnicodeDecodeError:
                        return None
        except:
            continue
    return None

def extract_main_text(raw_text):
    if not raw_text:
        return None
    start_marker = '*** START OF THIS PROJECT GUTENBERG EBOOK'
    end_marker = '*** END OF THIS PROJECT GUTENBERG EBOOK'
    start = raw_text.upper().find(start_marker)
    end = raw_text.upper().find(end_marker)
    if start != -1:
        raw_text = raw_text[start + len(start_marker):]
    if end != -1:
        raw_text = raw_text[:end]
    return raw_text.strip()

def clean_text(text):
    if not text:
        return ""
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\[\d+\]', '', text)
    return text.strip()[:500000]

def save_to_csv(books, filename="books_data.csv"):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ID', 'Title', 'Author', 'Summary', 'Text', 'Cover'])  # Cover أصبح آخر عمود
        for book in books:
            writer.writerow(book)
    print(f"\n📚 تم حفظ {len(books)} كتاب في الملف: {filename}")

# تحميل 10 كتب
books = []

for book_id in tqdm(range(1, 11), desc="📖 تحميل الكتب"):
    title, author, summary, cover_url = get_book_metadata(book_id)
    if not title:
        continue

    raw_text = get_book_text(book_id)
    if not raw_text:
        continue

    main_text = extract_main_text(raw_text)
    cleaned = clean_text(main_text)

    if not cleaned:
        continue

    books.append([book_id, title, author, summary, cleaned, cover_url])  # Cover أصبح آخر عنصر
    time.sleep(2)

# حفظهم في CSV
save_to_csv(books)

📖 تحميل الكتب:  10%|█         | 1/10 [00:30<04:32, 30.28s/it]

❌ Error fetching metadata for 1: HTTPSConnectionPool(host='www.gutenberg.org', port=443): Max retries exceeded with url: /ebooks/1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001A529EE6990>, 'Connection to www.gutenberg.org timed out. (connect timeout=30)'))


📖 تحميل الكتب:  40%|████      | 4/10 [01:48<02:43, 27.22s/it]

❌ Error fetching metadata for 4: HTTPSConnectionPool(host='www.gutenberg.org', port=443): Max retries exceeded with url: /ebooks/4 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001A529EE6850>, 'Connection to www.gutenberg.org timed out. (connect timeout=30)'))


📖 تحميل الكتب:  90%|█████████ | 9/10 [03:28<00:19, 19.31s/it]

❌ Error fetching metadata for 9: HTTPSConnectionPool(host='www.gutenberg.org', port=443): Max retries exceeded with url: /ebooks/9 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001A529EE7890>, 'Connection to www.gutenberg.org timed out. (connect timeout=30)'))


📖 تحميل الكتب: 100%|██████████| 10/10 [03:34<00:00, 21.44s/it]


📚 تم حفظ 6 كتاب في الملف: books_data.csv


In [1]:
import pandas as pd

# تحميل الملفين
df_main = pd.read_csv(r'C:/Users/BADAWY/Desktop/GP- Project/Data Set/Main_Dataset.csv')
df_with_rating = pd.read_csv(r'C:/Users/BADAWY/Desktop/GP- Project/Data Set/Main_Final_with_user_data.csv')

# إزالة الفراغات من أسماء الأعمدة
df_main.columns = df_main.columns.str.strip()
df_with_rating.columns = df_with_rating.columns.str.strip()

# استخراج فقط الأعمدة المطلوبة: id و Rating
rating_data = df_with_rating[['ID', 'Rating']].copy()
rating_data.rename(columns={'ID': 'id'}, inplace=True)

# دمج عمود Rating فقط بناءً على id
df_main_with_rating = df_main.merge(rating_data, on='id', how='left')

# حفظ الملف الجديد
df_main_with_rating.to_csv(r'C:/Users/BADAWY/Desktop/GP- Project/Data Set/Main_Dataset_with_rating.csv', index=False)

print("✅ تم إضافة عمود Rating فقط بنجاح.")


✅ تم إضافة عمود Rating فقط بنجاح.


# Read the data Set 

In [ ]:
ds = pd.read_excel('C:/Users/BADAWY/Desktop/GP- Project/Data Set/Main_Final.xlsx')
